<a href="https://colab.research.google.com/github/Cchancee/summative_MLOP/blob/main/summative_mlop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.datasets import fashion_mnist

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import fashion_mnist

# Load dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize
x_train = x_train / 255.0
x_test = x_test / 255.0

# Reshape to fit CNN input: (28, 28, 1)
x_train = x_train[..., np.newaxis]
x_test = x_test[..., np.newaxis]

# Class labels
class_names = [
    "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
    "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"
]


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Dropout(0.25),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])


datagen = ImageDataGenerator(
    rotation_range=15,
    zoom_range=0.1,
    horizontal_flip=True
)

datagen.fit(x_train)

history = model.fit(x_train, y_train, epochs=15, validation_split=0.1)


Epoch 1/15
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 33s 19ms/step - accuracy: 0.6974 - loss: 0.8362 - val_accuracy: 0.8548 - val_loss: 0.3819
Epoch 2/15
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 40s 18ms/step - accuracy: 0.8453 - loss: 0.4301 - val_accuracy: 0.8863 - val_loss: 0.3133
Epoch 3/15
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 30s 18ms/step - accuracy: 0.8628 - loss: 0.3781 - val_accuracy: 0.8913 - val_loss: 0.2996
Epoch 4/15
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 31s 18ms/step - accuracy: 0.8743 - loss: 0.3485 - val_accuracy: 0.9012 - val_loss: 0.2730
Epoch 5/15
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 42s 19ms/step - accuracy: 0.8823 - loss: 0.3230 - val_accuracy: 0.9030 - val_loss: 0.2643
Epoch 6/15
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 40s 18ms/step - accuracy: 0.8857 - loss: 0.3115 - val_accuracy: 0.9042 - val_loss: 0.2596
Epoch 7/15
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 41s 19ms/step - accuracy: 0.8901 - loss: 0.2944 - val_accuracy: 0.9053 - val_loss: 0.2517
Epoch 8/15
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 31s 18ms/step - accuracy: 0.8951 -

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc:.4f}")


313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9095 - loss: 0.2462
Test accuracy: 0.9109


In [ ]:
model.save("models/fashion_cnn.h5")